# ML for classifying descent sets of permutations

*Goal*: Make a classifier that can predict the descent sets of permutations in some medium-size symmetric group, say $S_{20}$.

*Model inputs*: I think that we should be able to represent a permutation in one-line notation, i.e. $[w(1), \ldots, w(n)]$.
We might also think of this as points in the orbit $S_n \cdot (1, 2, \ldots, n) \subseteq \mathbb{R}^n$.
Since we want all inputs to live inside the box $[-1, 1]^n$ for network initialisation reasons, we will scale each point by $1/n$.
We're also working in Python and so permuting the set $\{0, 1, \ldots, n-1\}$ will be easier.

*Model outputs*: A vector in $\mathbb{R}^{n-1}$, where each entry is treated like a binary classification of whether the transposition $s_i = (i, i+1)$ is a right descent or not.
The output units are logits, we'll apply the [sigmoid function](https://pytorch.org/docs/stable/generated/torch.nn.Sigmoid.html) $x \mapsto \frac{1}{(1 + e^{-x})}$ componentwise to go from that to a vector of probabilities.
So this network should be viewed as a function $\mathbb{R}^n \to \mathbb{R}^{n-1}$, where projecting to the $i$th output only works like a binary classifier $\mathbb{R}^n \to \mathbb{R}$.

Author: Joel Gibson<br>
Additions to handle left descent sets and permutation matrices by Geordie Williamson.

In [ ]:
from typing import List, Sequence, Set

import numpy as np # numbers, matrices, etc.

import pandas as pd # displaying data in a nice way
import torch # machine learning stuff
import torch.nn as nn # neural net library

from numpy.typing import NDArray

## Generate a dataset

We can use `np.random` functions to create random permutations.
We'll also be using `np.random.default_rng(seed)` with a `seed` argument to try to get reproducible results (at least reproducible on one machine, not necessarily another), which can be handy for tracking down bugs.

In [ ]:
# Create a random number generator with a specific seed, and then create two permutations.
# Note that we can reproduce these two "random" permutations if we re-seed the generator.
rand_gen = np.random.default_rng(seed=1)
print("First permutation:", rand_gen.permutation(5))
print("Second permutation:", rand_gen.permutation(7))

rand_gen = np.random.default_rng(seed=1)
print("Third permutation:", rand_gen.permutation(5))
print("Fourth permutation:", rand_gen.permutation(7))

First permutation: [4 0 1 2 3]
Second permutation: [6 3 5 4 0 1 2]
Third permutation: [4 0 1 2 3]
Fourth permutation: [6 3 5 4 0 1 2]


A convenience function for calculating right and left descent sets of permutations

In [ ]:
def right_descent_set(perm: Sequence[int]) -> Set[int]:
    """
    Return the right descent set, i.e. those i in the range [0, n-1) such that the transposition
    (i, i+1) multiplied on the right lowers the length of the permutation.

    Multiplication on the right by a transposition (i, j) corresponds to swapping perm[i] and perm[j],
    so we can simply check whether we are making a previously out-of-order pair into an in-order pair.

    >>> right_descent_set((0, 1, 2))
    set()
    >>> right_descent_set((2, 1, 0))
    {0, 1}
    """
    return {i for i in range(len(perm) - 1) if perm[i] > perm[i+1]}

def left_descent_set(perm: Sequence[int]) -> Set[int]:
    """
    Return the left descent set, i.e. those i in the range [0, n-1) such that the transposition
    (i, i+1) multiplied on the left lowers the length of the permutation.

    Multiplication on the left by a transposition (i, j) corresponds to swapping i and j in the
    sequence (perm[0],perm[1],...,perm[n-1]).

    so we can simply check whether i occurs to the right of i+1 in this sequence

    >>> left_descent_set((0, 1, 2))
    set()
    >>> right_descent_set((2, 1, 0))
    {0, 1}
    """
    perm = tuple(perm)
    return {i for i in range(len(perm) - 1) if perm.index(i) > perm.index(i+1)}

rand_gen = np.random.default_rng(seed=1)
perm = rand_gen.permutation(10)
print(perm, right_descent_set(perm))
print(perm, left_descent_set(perm))

[8 4 7 0 1 2 5 9 6 3] {0, 8, 2, 7}
[8 4 7 0 1 2 5 9 6 3] {3, 6, 7}


For fun we check the symmetry property:

In [ ]:
rand_gen = np.random.default_rng(seed=1)
perm = rand_gen.permutation(10)
inverse_perm = np.argsort(perm)
print(right_descent_set(perm) == left_descent_set(inverse_perm))
print(left_descent_set(perm) == right_descent_set(inverse_perm))

True
True


Let's say that $n$ is the size of the symmetric group, i.e. $S_n$, and $L$ is the number of random permutations we want to generate. Our input tensor should be of shape $(L, n)$ and the output tensor should be of shape $(L, n-1)$. We can wrap these up with a [torch.utils.data.TensorDataset](https://pytorch.org/docs/stable/data.html?highlight=tensordataset#torch.utils.data.TensorDataset), which will have utilities for shuffling these two tensors in parallel during training.

In [ ]:
def generate_dataset(gen: np.random.Generator, n: int, L: int, sidedness: str):
    # Generate permutations of {0, ..., n-1}: tensor shape (L, n).
    permutations = np.stack([gen.permutation(n) for _ in range(L)])

    # Make a (L, n-1) shape tensor of zeros, which we will fill up with the descent sets
    descent_sets = np.zeros((L, n-1))
    for i in range(L):
        if sidedness == 'left':
          for s in left_descent_set(permutations[i]):
              descent_sets[i, s] = 1
        if sidedness == 'right':
          for s in right_descent_set(permutations[i]):
              descent_sets[i, s] = 1

    # Scale the permutations so that they are between 0 and 1, and convert to Pytorch tensors.
    return torch.utils.data.TensorDataset(
        torch.from_numpy(permutations).float() / n,
        torch.from_numpy(descent_sets).float()
    )

# Once created, items can be pulled out of the dataset by indexing [...].
# Let's check that one of the rows in the dataset looks correct.
dataset = generate_dataset(rand_gen, 20, 1000,'left')
dataset[1]

(tensor([0.4500, 0.6000, 0.2500, 0.0000, 0.7000, 0.3000, 0.1000, 0.4000, 0.1500,
         0.8000, 0.5000, 0.9500, 0.3500, 0.0500, 0.9000, 0.6500, 0.5500, 0.7500,
         0.8500, 0.2000]),
 tensor([0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1.,
         1.]))

## Network architecture

We'll do a fully-connected network with a configurable number of internal layers, followed by a component-wise sigmoid at the end.
I.e. the output of this model is already in probability space rather than logits space.

In [ ]:
class FCModel(nn.Module):
    def __init__(self, n: int, layer_dims: List[int]):
        super().__init__()
        self.n = n
        self.layer_dims = layer_dims

        self.layers = nn.ModuleList([
            nn.Linear(in_dim, out_dim)
            for in_dim, out_dim in zip([n, *layer_dims], [*layer_dims, n - 1])
        ])

    def forward(self, x: torch.tensor):
        for i, layer in enumerate(self.layers):
            x = layer(x)

            # We don't want to ReLU the last layer
            if i != len(self.layers) - 1:
                x = nn.functional.relu(x)

        return torch.sigmoid(x)

# Test our model has all its sizes etc set up right.
model = FCModel(n=20, layer_dims=[100, 10, 5])
model(torch.zeros((100, 20))).shape                   # 'batch variable!'

torch.Size([100, 19])

Let's also make a function to count how many predictions the network is making correctly, with some cutoff probability for whether we determine something to be in or out of the set.
We count a prediction correct only if it is absolutely the same as the other set.
Something which is off by only one element is counted as incorrect.

In [ ]:
def count_correct(model, dataset, cutoff=0.5):
    perms, expected = dataset[:]
    with torch.no_grad():
        classification = model(perms) >= cutoff

    total = perms.shape[0]
    correct = 0
    for i in range(total):
        if torch.all(classification[i] == expected[i]):
            correct += 1

    return correct, total

# We would expect a new randomly initialised network to get nothing correct.
count_correct(model, dataset)

(0, 1000)

## Training hyperparameters

The only thing of real note here is the fact that I'm using the binary cross-entropy loss [BCELoss](https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html), since I want to treat each output as its own binary classification.

In [ ]:
# Size of symmetric group.
n = 35

# Train and test data: by setting the random seed here, we get the same data each time.
rand_gen = np.random.default_rng(seed=1)
train_data = generate_dataset(rand_gen, n, 20_000,'right')
test_data = generate_dataset(rand_gen, n, 5_000,'right')

# Model shape. Note that PyTorch will randomise the initial model parameters each time here.
model = FCModel(n, [500, 100])       # can experiment here!!

# Optimisation method.
optimiser = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.5) # can mess with learning rate if you want

# Loss function.
loss_fn = nn.BCELoss() # binary cross entropy

# Epochs to use for training. The whole training dataset will be seen once during each epoch.
epochs = 100

# Minibatch size: the error over a minibatch will be accumulated before a gradient step.
# There are multiple minibatches within an epoch.
minibatch_size = 50

## Training

First we see that the network learns the right descent set easily (in 100 epochs or so).

(Remember that one should rerun the previous cell each time one tries a new experiment, otherwise the network weights are preserved.)

In [ ]:
train_loss = np.zeros(epochs)
test_loss = np.zeros(epochs)

for epoch in range(epochs):
    for perms, expected in torch.utils.data.DataLoader(train_data, batch_size=minibatch_size, shuffle=True):
        optimiser.zero_grad()
        output = model(perms)
        loss = loss_fn(output, expected)
        loss.backward()
        optimiser.step()

    with torch.no_grad():
        perms, expected = train_data[:]
        train_loss[epoch] = float(loss_fn(model(perms), expected))

        perms, expected = test_data[:]
        test_loss[epoch] = float(loss_fn(model(perms), expected))

    if epoch % 10 == 9:
        correct, total = count_correct(model, test_data, cutoff=0.5)
        print(f"Epoch {epoch:4}: Train loss {train_loss[epoch]:.2f}, Test loss {test_loss[epoch]:.2f}, {correct:5} out of {total:5} correct ({correct/total:.2%})")

Epoch    9: Train loss 0.12, Test loss 0.12,  2086 out of  5000 correct (41.72%)
Epoch   19: Train loss 0.05, Test loss 0.05,  4012 out of  5000 correct (80.24%)
Epoch   29: Train loss 0.03, Test loss 0.04,  4483 out of  5000 correct (89.66%)
Epoch   39: Train loss 0.02, Test loss 0.03,  4871 out of  5000 correct (97.42%)
Epoch   49: Train loss 0.02, Test loss 0.02,  4968 out of  5000 correct (99.36%)
Epoch   59: Train loss 0.02, Test loss 0.02,  4949 out of  5000 correct (98.98%)
Epoch   69: Train loss 0.01, Test loss 0.01,  4836 out of  5000 correct (96.72%)
Epoch   79: Train loss 0.01, Test loss 0.01,  4985 out of  5000 correct (99.70%)
Epoch   89: Train loss 0.01, Test loss 0.01,  4987 out of  5000 correct (99.74%)
Epoch   99: Train loss 0.01, Test loss 0.01,  4990 out of  5000 correct (99.80%)


Now we switch to left descent sets, and see that the network struggles much more to train.

One does get some evidence that it can learn the left descent set if one lowers n and lets it train much more. However the difference between the two tasks is really striking.

In [ ]:
train_data = generate_dataset(rand_gen, n, 20_000,'left')
test_data = generate_dataset(rand_gen, n, 5_000,'left')

# Model shape. Note that PyTorch will randomise the initial model parameters each time here.
model = FCModel(n, [500, 100])       # can experiment here!!

train_loss = np.zeros(epochs)
test_loss = np.zeros(epochs)

for epoch in range(epochs):
    for perms, expected in torch.utils.data.DataLoader(train_data, batch_size=minibatch_size, shuffle=True):
        optimiser.zero_grad()
        output = model(perms)
        loss = loss_fn(output, expected)
        loss.backward()
        optimiser.step()

    with torch.no_grad():
        perms, expected = train_data[:]
        train_loss[epoch] = float(loss_fn(model(perms), expected))

        perms, expected = test_data[:]
        test_loss[epoch] = float(loss_fn(model(perms), expected))

    if epoch % 10 == 9:
        correct, total = count_correct(model, test_data, cutoff=0.5)
        print(f"Epoch {epoch:4}: Train loss {train_loss[epoch]:.2f}, Test loss {test_loss[epoch]:.2f}, {correct:5} out of {total:5} correct ({correct/total:.2%})")

Epoch    9: Train loss 0.69, Test loss 0.69,     0 out of  5000 correct (0.00%)
Epoch   19: Train loss 0.69, Test loss 0.69,     0 out of  5000 correct (0.00%)
Epoch   29: Train loss 0.69, Test loss 0.69,     0 out of  5000 correct (0.00%)
Epoch   39: Train loss 0.69, Test loss 0.69,     0 out of  5000 correct (0.00%)
Epoch   49: Train loss 0.69, Test loss 0.69,     0 out of  5000 correct (0.00%)
Epoch   59: Train loss 0.69, Test loss 0.69,     0 out of  5000 correct (0.00%)
Epoch   69: Train loss 0.69, Test loss 0.69,     0 out of  5000 correct (0.00%)
Epoch   79: Train loss 0.69, Test loss 0.69,     0 out of  5000 correct (0.00%)
Epoch   89: Train loss 0.69, Test loss 0.69,     0 out of  5000 correct (0.00%)
Epoch   99: Train loss 0.69, Test loss 0.69,     0 out of  5000 correct (0.00%)


## Modifying the model to accept permutation matrices.

Now we change the model slightly to accept permutation matrices.

This allows us to test our hypothesis that the stark difference between left and right descent sets is due to the representation.

In [ ]:
def flat_permutation_matrix(perm:tuple) -> NDArray:
  n = len(perm)
  mat = np.zeros((n,n))
  for i, v in enumerate(perm): mat[i,v] = 1
  return mat.reshape((n**2))

def generate_dataset_matrices(gen: np.random.Generator, n: int, L: int,sidedness:str):
    # Generate permutations of {0, ..., n-1}: tensor shape (L, n).
    permutations = np.stack([gen.permutation(n) for _ in range(L)])

    permutation_mats = np.stack([flat_permutation_matrix(p) for p in permutations])

    # Make a (L, n-1) shape tensor of zeros, which we will fill up with the descent sets
    descent_sets = np.zeros((L, n-1))
    for i in range(L):
        if sidedness == 'left':
          for s in left_descent_set(permutations[i]):
              descent_sets[i, s] = 1
        if sidedness == 'right':
          for s in right_descent_set(permutations[i]):
              descent_sets[i, s] = 1

    # Scale the permutations so that they are between 0 and 1, and convert to Pytorch tensors.
    return torch.utils.data.TensorDataset(
        torch.from_numpy(permutation_mats).float(),
        torch.from_numpy(descent_sets).float()
    )

# Once created, items can be pulled out of the dataset by indexing [...].
# Let's check that one of the rows in the dataset looks correct.
dataset = generate_dataset_matrices(rand_gen, 20, 1000,'left')
dataset[1]

[1. 0. 0. 0. 0. 1. 0. 1. 0.]


(tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.,

In [ ]:
class FCModel_matrices(nn.Module):
    def __init__(self, n: int, layer_dims: List[int]):
        super().__init__()
        self.n = n
        self.layer_dims = layer_dims

        self.layers = nn.ModuleList([
            nn.Linear(in_dim, out_dim)
            for in_dim, out_dim in zip([n**2, *layer_dims], [*layer_dims, n - 1])
        ])

    def forward(self, x: torch.tensor):
        for i, layer in enumerate(self.layers):
            x = layer(x)

            # We don't want to ReLU the last layer
            if i != len(self.layers) - 1:
                x = nn.functional.relu(x)

        return torch.sigmoid(x)

# Test our model has all its sizes etc set up right.
model = FCModel_matrices(n=20, layer_dims=[100, 10, 5])
model(torch.zeros((100, 20**2))).shape                   # 'batch variable!'

torch.Size([100, 19])

In [ ]:
def count_correct(model, dataset, cutoff=0.5):
    perms, expected = dataset[:]
    with torch.no_grad():
        classification = model(perms) >= cutoff

    total = perms.shape[0]
    correct = 0
    for i in range(total):
        if torch.all(classification[i] == expected[i]):
            correct += 1

    return correct, total

# We would expect a new randomly initialised network to get nothing correct.
count_correct(model, dataset)

(0, 1000)

In [ ]:
# Size of symmetric group.
# Note: we make this smaller than above.

n = 20

# Train and test data: by setting the random seed here, we get the same data each time.
rand_gen = np.random.default_rng(seed=1)
train_data = generate_dataset_matrices(rand_gen, n, 20_000,'left')
test_data = generate_dataset_matrices(rand_gen, n, 5_000,'left')

# Model shape. Note that PyTorch will randomise the initial model parameters each time here.
model = FCModel_matrices(n, [1000, 100])       # can experiment here!!

# Optimisation method.
optimiser = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.5) # can mess with learning rate if you want

# Loss function.
loss_fn = nn.BCELoss() # binary cross entropy

# Epochs to use for training. The whole training dataset will be seen once during each epoch.
epochs = 100

# Minibatch size: the error over a minibatch will be accumulated before a gradient step.
# There are multiple minibatches within an epoch.
minibatch_size = 50

Note, in comparison to our earlier experiments, the input dimension is now $n^2$-dimensional, rather than $n$-dimensional. Hence we probably need a larger first hidden layer to accommodate this. We did this by replacing 500 by 1000 above. This leads to slightly longer training times.

If one runs the previous cell and the next, one sees (almost) no difference between 'left' and 'right' descents. Also accuracy is very good after 100 epochs of training. This appears to confirm the hypothesis that the representation really was the issue in our previous experiments.

In [ ]:
train_loss = np.zeros(epochs)
test_loss = np.zeros(epochs)

for epoch in range(epochs):
    for perms, expected in torch.utils.data.DataLoader(train_data, batch_size=minibatch_size, shuffle=True):
        optimiser.zero_grad()
        output = model(perms)
        loss = loss_fn(output, expected)
        loss.backward()
        optimiser.step()

    with torch.no_grad():
        perms, expected = train_data[:]
        train_loss[epoch] = float(loss_fn(model(perms), expected))

        perms, expected = test_data[:]
        test_loss[epoch] = float(loss_fn(model(perms), expected))

    if epoch % 10 == 9 or epoch < 10:
        correct, total = count_correct(model, test_data, cutoff=0.5)
        print(f"Epoch {epoch:4}: Train loss {train_loss[epoch]:.2f}, Test loss {test_loss[epoch]:.2f}, {correct:5} out of {total:5} correct ({correct/total:.2%})")

Epoch    0: Train loss 0.69, Test loss 0.69,     0 out of  5000 correct (0.00%)
Epoch    1: Train loss 0.69, Test loss 0.69,     2 out of  5000 correct (0.04%)
Epoch    2: Train loss 0.68, Test loss 0.68,     7 out of  5000 correct (0.14%)
Epoch    3: Train loss 0.65, Test loss 0.65,    49 out of  5000 correct (0.98%)
Epoch    4: Train loss 0.55, Test loss 0.55,    88 out of  5000 correct (1.76%)
Epoch    5: Train loss 0.39, Test loss 0.40,   210 out of  5000 correct (4.20%)
Epoch    6: Train loss 0.28, Test loss 0.29,   602 out of  5000 correct (12.04%)
Epoch    7: Train loss 0.21, Test loss 0.21,  1210 out of  5000 correct (24.20%)
Epoch    8: Train loss 0.16, Test loss 0.17,  1813 out of  5000 correct (36.26%)
Epoch    9: Train loss 0.13, Test loss 0.14,  2259 out of  5000 correct (45.18%)
Epoch   19: Train loss 0.04, Test loss 0.04,  4322 out of  5000 correct (86.44%)
Epoch   29: Train loss 0.02, Test loss 0.02,  4785 out of  5000 correct (95.70%)
Epoch   39: Train loss 0.01, Test 